In [1]:
import numpy as np
import pandas as pd

# 任务1.1结合业务对数据进行探索与预处理

In [2]:
# 读取会员信息表与销售流水表数据
member = pd.read_excel('项目数据1.xlsx')
order = pd.read_csv('项目数据2.csv')

/home/linux/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
member.head()

,kh,csrq,xb,djsj
0,c68b20b4,2002-11-02 00:00:00,0.0,2013-05-11 00:00:00.000
1,1ca15332,NaN,0.0,2004-11-04 16:31:52.436
2,a37cc182,1967-02-17 00:00:00,0.0,2004-12-31 21:24:34.216
3,2ab88539,1982-06-01 00:00:00,0.0,2010-11-19 00:00:00.000
4,b4c77269,1964-02-05 00:00:00,0.0,2007-12-14 00:00:00.000


In [4]:
order.head()

,kh,dtime,spbm,sl,sj,je,spmc,jf,syjh,djh,gzbm,gzmc
0,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,290.0,270.20,兰芝化妆品正价瓶,270.20,6,25bb,8077.0,兰芝柜
1,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,325.0,302.80,兰芝化妆品正价瓶,302.80,6,25bb,8077.0,兰芝柜
2,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,195.0,181.80,兰芝化妆品正价瓶,181.80,6,25bb,8077.0,兰芝柜
3,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,270.0,251.55,兰芝化妆品正价瓶,251.55,6,25bb,8077.0,兰芝柜
4,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,2,245.0,456.55,兰芝化妆品正价瓶,456.55,6,25bb,8077.0,兰芝柜


## 检测与处理重复值

In [5]:
print('卡号的重复值有：',len(member['kh']) - len(set(member['kh'].values)))

卡号的重复值有： 6


In [6]:
# 去除重复的卡号
member.drop_duplicates(subset = 'kh', inplace = True)

In [7]:
print('卡号的重复值有：',len(member['kh']) - len(set(member['kh'].values)))

卡号的重复值有： 0


## 检测与处理缺失值

In [8]:
member.isnull().sum()

kh          0
csrq    34187
xb       9435
djsj    12684
dtype: int64

会员信息表存在信息缺失值

In [9]:
order.isnull().sum()

kh       1017486
dtime          0
spbm           0
sl             0
sj             0
je             0
spmc           0
jf       1017486
syjh           0
djh            0
gzbm     1017486
gzmc     1036956
dtype: int64

销售流水表包含非会员购买的订单，卡号缺失为非会员的订单

将会员信息表含有缺失值的记录删除

In [10]:
#去除空值
member2 = member.dropna()
#重置索引
member2.reset_index(inplace = True)
member2.drop(axis = 1,columns = 'index',inplace = True)
member2.isnull().sum()

/home/linux/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


kh      0
csrq    0
xb      0
djsj    0
dtype: int64

## 检测与处理异常值

日期异常值

In [11]:
# 首先删除会员信息表和销售流水表中，日期因为越界异常而无法转换为datetime格式的记录
def filter_date(datestr):
    try:
        pd.to_datetime(datestr)
        return True
    except:
        return False

In [12]:
# 过滤出生日期越界记录
mem_csrq_filter = member2['csrq'].apply(lambda x:filter_date(x))
member3 = member2[mem_csrq_filter]

In [13]:
# 过滤登记时间越接记录
mem_djsj_filter = member3['djsj'].apply(lambda x:filter_date(x))
member3 = member3[mem_djsj_filter]

In [14]:
# 重置索引
member3.reset_index(inplace = True)
member3.drop(axis = 1,columns = 'index',inplace = True)

In [15]:
member3.head()

,kh,csrq,xb,djsj
0,c68b20b4,2002-11-02 00:00:00,0.0,2013-05-11 00:00:00.000
1,a37cc182,1967-02-17 00:00:00,0.0,2004-12-31 21:24:34.216
2,2ab88539,1982-06-01 00:00:00,0.0,2010-11-19 00:00:00.000
3,b4c77269,1964-02-05 00:00:00,0.0,2007-12-14 00:00:00.000
4,83a91070,1968-06-18 00:00:00,1.0,2004-05-01 16:50:13.950


In [16]:
# 过滤销售流水表中销售日期越界记录
ord_dtime_filter = order['dtime'].apply(lambda x:filter_date(x))
order2 = order[ord_dtime_filter]
# 重置索引
order2.reset_index(inplace = True)
order2.drop(axis = 1,columns = 'index',inplace = True)

In [17]:
order2.head()

,kh,dtime,spbm,sl,sj,je,spmc,jf,syjh,djh,gzbm,gzmc
0,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,290.0,270.20,兰芝化妆品正价瓶,270.20,6,25bb,8077.0,兰芝柜
1,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,325.0,302.80,兰芝化妆品正价瓶,302.80,6,25bb,8077.0,兰芝柜
2,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,195.0,181.80,兰芝化妆品正价瓶,181.80,6,25bb,8077.0,兰芝柜
3,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,270.0,251.55,兰芝化妆品正价瓶,251.55,6,25bb,8077.0,兰芝柜
4,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,2,245.0,456.55,兰芝化妆品正价瓶,456.55,6,25bb,8077.0,兰芝柜


In [18]:
# 将日期数据转换为datetime格式
member3['csrq'] = pd.to_datetime(member3['csrq'])
member3['djsj'] = pd.to_datetime(member3['djsj'])
order2['dtime'] = pd.to_datetime(order2['dtime'])

In [19]:
# 探索日期数据有无业务上的异常值
#出生日期
csrq_yearlist = member3['csrq'].apply(lambda x:x.year)

print('出生年份最大值与最小值:',np.max(csrq_yearlist),np.min(csrq_yearlist))

出生年份最大值与最小值: 2195 1753


显然出生年份存在异常值，需要过滤,设定正常出生日期介于1920年-2010年

In [20]:
csrq_nor = member3['csrq'].apply(lambda x:x.year <= 2010 and x.year >= 1920)
member3 = member3[csrq_nor]
# 重置索引
member3.reset_index(inplace = True)
member3.drop(axis = 1,columns = 'index',inplace = True)

In [21]:
csrq_yearlist_mod = member3['csrq'].apply(lambda x:x.year)
print('出生年份最大值与最小值:',np.max(csrq_yearlist_mod),np.min(csrq_yearlist_mod))

出生年份最大值与最小值: 2010 1920


In [22]:
#入会时间
djsj_yearlist = member3['djsj'].apply(lambda x:x.year)

In [23]:
print('入会时间最大值与最小值:',np.max(djsj_yearlist),np.min(djsj_yearlist))

入会时间最大值与最小值: 2080 1900


入会时间同样存在异常数据

In [29]:
member3[djsj_yearlist < 2000]

,kh,csrq,xb,djsj
105900,91d64726,1967-02-18,0.0,1900-01-01


In [30]:
member3[djsj_yearlist > 2019]

,kh,csrq,xb,djsj
8237,e26e252c,1978-10-22,0.0,2080-07-03
107182,b66b4508,1981-10-17,0.0,2080-01-17


In [31]:
djsj_nor = member3['djsj'].apply(lambda x:x.year <= 2019 and x.year >= 2000)
member3 = member3[djsj_nor]
# 重置索引
member3.reset_index(inplace = True)
member3.drop(axis = 1,columns = 'index',inplace = True)

In [32]:
djsj_yearlist_mod = member3['djsj'].apply(lambda x:x.year)
print('入会时间最大值与最小值:',np.max(djsj_yearlist_mod),np.min(djsj_yearlist_mod))

入会时间最大值与最小值: 2018 2002


正常入会时间，最早为2002，最晚为2018

In [33]:
# 销售流水表
dtime_yearlist = order2['dtime'].apply(lambda x:x.year)

In [34]:
print('销售年份时间最大值与最小值:',np.max(dtime_yearlist),np.min(dtime_yearlist))

销售年份时间最大值与最小值: 2018 2015


销售年份时间正常

处理销售金额异常数据

In [35]:
order2.describe()

,sl,sj,je,jf,syjh,gzbm
count,1.893532e+06,1.893532e+06,1.893532e+06,876046.000000,1.893532e+06,876046.000000
mean,1.060906e+00,1.093785e+03,1.047361e+03,1035.463948,1.002693e+02,6328.248018
std,2.483580e+00,2.582808e+03,2.697462e+03,2064.978604,6.689405e+01,2371.230147
min,-3.810000e+02,1.000000e-01,-6.712575e+05,-201616.000000,1.000000e+00,0.000000
25%,1.000000e+00,3.200000e+02,3.100000e+02,180.000000,8.700000e+01,4334.000000
50%,1.000000e+00,5.700000e+02,5.640000e+02,510.000000,9.400000e+01,8064.000000
75%,1.000000e+00,1.100000e+03,1.098000e+03,1259.000000,1.010000e+02,8123.000000
max,1.492000e+03,1.342515e+06,1.342515e+06,268503.000000,3.200000e+02,9005.000000


销售数量,销售金额和积分存在负值,将其删除

In [25]:
order3 = order2[order2['sl'] > 0]
order3 = order3[order3['je'] >= 0]
#积分非会员的部分为NaN，防止将其误删
order3 = order3[order3['jf'].apply(lambda x:not(x < 0))]
# 重置索引
order3.reset_index(inplace = True)
order3.drop(axis = 1,columns = 'index',inplace = True)

In [26]:
#print(len(order2),len(order3))
print('删除的销售流水表异常数据量为：%d'%(len(order2) -len(order3)))

删除的销售流水表异常数据量为：26884


In [38]:
order3.describe()

,sl,sj,je,jf,syjh,gzbm
count,1.866648e+06,1.866648e+06,1.866648e+06,869012.000000,1.866648e+06,869012.000000
mean,1.098031e+00,1.086410e+03,1.088572e+03,1060.017791,1.000142e+02,6337.096189
std,2.305768e+00,2.473873e+03,2.542066e+03,2008.289584,6.672042e+01,2368.315911
min,1.000000e+00,1.000000e-01,0.000000e+00,0.000000,1.000000e+00,0.000000
25%,1.000000e+00,3.200000e+02,3.200000e+02,192.000000,8.700000e+01,4334.000000
50%,1.000000e+00,5.700000e+02,5.750000e+02,520.000000,9.400000e+01,8064.000000
75%,1.000000e+00,1.100000e+03,1.103000e+03,1272.000000,1.010000e+02,8123.000000
max,1.492000e+03,1.342515e+06,1.342515e+06,268503.000000,3.200000e+02,9005.000000


In [39]:
order3[order3['je'] == np.max(order3['je'])]

,kh,dtime,spbm,sl,sj,je,spmc,jf,syjh,djh,gzbm,gzmc
1555375,11a77271,2017-07-03 13:26:11.250,d72de91e,1,1342515.0,1342515.0,欧米伽.,268503.0,101,0e7e,8078.0,瑞士钟表柜


保存预处理的会员信息表与销售流水表

In [40]:
member3.to_csv('temp/member_pre.csv',index = False)
order3.to_csv('temp/order_pre.csv',index = False)

## 任务1.2将会员信息表与销售流水表关联合并

In [41]:
task1 = pd.merge(member3, order3, how = 'outer', on = 'kh')

In [42]:
#包含本店入会会员的销售记录，非本门店入会会员的销售记录（无入会信息），非会员销售记录（无卡号记录）
task1.head()

,kh,csrq,xb,djsj,dtime,spbm,sl,sj,je,spmc,jf,syjh,djh,gzbm,gzmc
0,c68b20b4,2002-11-02,0.0,2013-05-11 00:00:00.000,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a37cc182,1967-02-17,0.0,2004-12-31 21:24:34.216,2016-09-25 10:49:14.016,d62a69e3,1.0,499.0,297.07,WMF D无,297.08,101.0,7cd8,7296.0,WMF 柜
2,a37cc182,1967-02-17,0.0,2004-12-31 21:24:34.216,2016-09-25 10:49:14.016,d62a69e3,1.0,12288.0,11089.93,WMF D无,11089.92,101.0,7cd8,7296.0,WMF 柜
3,a37cc182,1967-02-17,0.0,2004-12-31 21:24:34.216,2017-02-28 17:17:35.533,252403ef,1.0,598.0,598.00,双立人商品 F无,598.00,102.0,ed0a,7242.0,双立人柜
4,2ab88539,1982-06-01,0.0,2010-11-19 00:00:00.000,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
len(task1)

1971598

In [44]:
task1.to_csv('temp/task1.csv',index = False)